In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
text= open("/content/drive/MyDrive/ml_data/Ramayana.txt","r").read()
print(text[:])

Ayodhya was a magnificent city on the banks of the river Sarayu in
Kosala Country .It had wide roads, huge buildings, beautiful parks
and glittering shops. The people of the city lived a happy and
contented life as they were ruled by a wonderful king called
Dasharatha. He cared for his people very deeply.
King Dasahratha had three wives, Kaushalya, Sumitra and Kaikeyi.
Kaushalya was the eldest queen. Though the king loved all his wives
deeply, it was Kaikeyi, his youngest queen who was his favourite.But
in spite of leading such a good life. Dasahratha was still an unhappy
man.This was because he had no children. He was getting old and
had no son to ascend to the throne.
Unable to see the king being sad, the sages advised him to
undertake a yagna and pray for children. Heeding to their advice, he
performed the yagna and pleased with him, Agni, the fire God,
emerged out of the fire pit, handed over a cup containing sweetened
milk to the king and told him "O king! This kheer is the nectar

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [5]:
text=text.replace('.',' <eos>. ')
text=text.replace('\n',' ')
text=text.replace('  ',' ')
text

'Ayodhya was a magnificent city on the banks of the river Sarayu in Kosala Country <eos>. It had wide roads, huge buildings, beautiful parks and glittering shops <eos>. The people of the city lived a happy and contented life as they were ruled by a wonderful king called Dasharatha <eos>. He cared for his people very deeply <eos>. King Dasahratha had three wives, Kaushalya, Sumitra and Kaikeyi <eos>. Kaushalya was the eldest queen <eos>. Though the king loved all his wives deeply, it was Kaikeyi, his youngest queen who was his favourite <eos>. But in spite of leading such a good life <eos>. Dasahratha was still an unhappy man <eos>. This was because he had no children <eos>. He was getting old and had no son to ascend to the throne <eos>. Unable to see the king being sad, the sages advised him to undertake a yagna and pray for children <eos>. Heeding to their advice, he performed the yagna and pleased with him, Agni, the fire God, emerged out of the fire pit, handed over a cup containin

In [6]:
tokenizer = Tokenizer()

In [7]:
tokenizer.fit_on_texts([text])

In [8]:
len(tokenizer.word_index)

2356

In [9]:
import string
text= text.replace('\n'," ")
split_text = text.split('\n')
clean_text = "".join(split_text)
clean_text.replace('  ',' ')
tokens= clean_text.split('.')
tokens= [' ' if w in string.punctuation else w for w in tokens]
Text=[]

for s in tokens:
  Text.append(s.lstrip())

print(Text)

['Ayodhya was a magnificent city on the banks of the river Sarayu in Kosala Country <eos>', 'It had wide roads, huge buildings, beautiful parks and glittering shops <eos>', 'The people of the city lived a happy and contented life as they were ruled by a wonderful king called Dasharatha <eos>', 'He cared for his people very deeply <eos>', 'King Dasahratha had three wives, Kaushalya, Sumitra and Kaikeyi <eos>', 'Kaushalya was the eldest queen <eos>', 'Though the king loved all his wives deeply, it was Kaikeyi, his youngest queen who was his favourite <eos>', 'But in spite of leading such a good life <eos>', 'Dasahratha was still an unhappy man <eos>', 'This was because he had no children <eos>', 'He was getting old and had no son to ascend to the throne <eos>', 'Unable to see the king being sad, the sages advised him to undertake a yagna and pray for children <eos>', 'Heeding to their advice, he performed the yagna and pleased with him, Agni, the fire God, emerged out of the fire pit, ha

In [10]:
input_sequences= []
for sentence in Text:
  tokenized_sentence= tokenizer.texts_to_sequences([sentence])[0]

  for i in range(1,len(tokenized_sentence)):
    input_sequences.append(tokenized_sentence[:i+1])

In [11]:
input_sequences[2]

[46, 9, 8, 1213]

In [12]:
max_len = max([len(x) for x in input_sequences])

In [13]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_input_sequences= pad_sequences(input_sequences, maxlen= max_len,padding='pre')
padded_input_sequences

array([[   0,    0,    0, ...,    0,   46,    9],
       [   0,    0,    0, ...,   46,    9,    8],
       [   0,    0,    0, ...,    9,    8, 1213],
       ...,
       [   0,    0,    0, ...,   31, 2355,  434],
       [   0,    0,    0, ..., 2355,  434, 2356],
       [   0,    0,    0, ...,  434, 2356,    1]], dtype=int32)

In [14]:
X= padded_input_sequences[:,:-1]
X

array([[   0,    0,    0, ...,    0,    0,   46],
       [   0,    0,    0, ...,    0,   46,    9],
       [   0,    0,    0, ...,   46,    9,    8],
       ...,
       [   0,    0,    0, ...,    4,   31, 2355],
       [   0,    0,    0, ...,   31, 2355,  434],
       [   0,    0,    0, ..., 2355,  434, 2356]], dtype=int32)

In [15]:
Y= padded_input_sequences[:,-1]
Y

array([   9,    8, 1213, ...,  434, 2356,    1], dtype=int32)

In [16]:
X.shape

(15576, 52)

In [17]:
Y.shape

(15576,)

In [18]:
from tensorflow.keras.utils import to_categorical
y= to_categorical(Y,num_classes=2357) #len(word_index)+1 (+1 as indexing starts from 1 in word_index and from 0 in OHE)


In [19]:
y.shape

(15576, 2357)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional

In [21]:
model= Sequential()
model.add(Embedding(2357,1000,input_length=52))
model.add(Bidirectional(LSTM(400)))
model.add(Dense(300, activation='relu'))
model.add(Dense(2357, activation='softmax'))

In [22]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 52, 1000)          2357000   
                                                                 
 bidirectional (Bidirection  (None, 800)               4483200   
 al)                                                             
                                                                 
 dense (Dense)               (None, 300)               240300    
                                                                 
 dense_1 (Dense)             (None, 2357)              709457    
                                                                 
Total params: 7789957 (29.72 MB)
Trainable params: 7789957 (29.72 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
model.fit(X,y,epochs=50)

Epoch 1/50
487/487 [==============================] - 34s 56ms/step - loss: 6.1407 - accuracy: 0.0779
Epoch 2/50
487/487 [==============================] - 13s 28ms/step - loss: 5.5561 - accuracy: 0.1150
Epoch 3/50
487/487 [==============================] - 11s 23ms/step - loss: 5.0976 - accuracy: 0.1516
Epoch 4/50
487/487 [==============================] - 12s 24ms/step - loss: 4.7061 - accuracy: 0.1801
Epoch 5/50
487/487 [==============================] - 11s 23ms/step - loss: 4.3357 - accuracy: 0.2097
Epoch 6/50
487/487 [==============================] - 11s 23ms/step - loss: 3.9506 - accuracy: 0.2422
Epoch 7/50
487/487 [==============================] - 11s 23ms/step - loss: 3.5530 - accuracy: 0.2794
Epoch 8/50
487/487 [==============================] - 11s 23ms/step - loss: 3.1210 - accuracy: 0.3294
Epoch 9/50
487/487 [==============================] - 11s 23ms/step - loss: 2.6507 - accuracy: 0.3929
Epoch 10/50
487/487 [==============================] - 11s 23ms/step - loss: 2.147

In [31]:
import time
import  numpy as np
text = "Dasahratha had three"
input_text= text

flag=0

while True:
  # tokenize
  token_text = tokenizer.texts_to_sequences([text])[0]
  # padding
  padded_token_text = pad_sequences([token_text], maxlen=52, padding='pre')
  # predict
  pos = np.argmax(model.predict(padded_token_text))

  for word,index in tokenizer.word_index.items():
    if index == pos:
      if(word=='eos'):
        text=text+"."
        flag=1
        break
      text= text+" "+word

  if(flag==1):
      break

print("Input: ",input_text)
print("Output: ",text)


1/1 [==============================] - 0s 20ms/step
Input:  Dasahratha had three
Output:  Dasahratha had three wives kaushalya sumitra and kaikeyi.


In [34]:
model.save('/content/drive/MyDrive/next_word_predictor.h5')